# Skull AI

The objective here is to make AI/Bots that can play the deception board/tile game: Skull. See this link [LINK] to a video showing how to play and then the flow diagram in the gitrepo showing the decision flow for  a player.

### Library imports

In [115]:
import random

### Player and Game classes

Player class.
Decisions for the bot to make:
 - To bet or place another tile
     - binary classifications
 - What tile to place
     - binary classification
 - what to bet
     - multiclass classification/ integer prediction
 - to bet or pass
     - binary classification
 - who to pick when seeking
     - multiclass classification
 
Defining a class for the player:
 - a player needs to be able to place a flower and skull [DONE]
 - a player needs to be able to remove a tile
     - randomly if they're skulled by somone else [DONE]
     - choose if they skull themselves
 - 

In [124]:
class skullPlayer:
    def __init__(self):
        self.flowers=3
        self.skull=1
        self.totalTiles=4
        self.hasSkull=True
        self.placedTiles=[]
        self.point=0
        
    def summary(self):
        print("Flowers: {}\nSkull: {}\nPoint: {}".format(self.flowers,self.skull,self.point))
    
    def placeFlower(self):
        if len(self.placedTiles)<self.totalTiles:
            if (len(self.placedTiles)-sum(self.placedTiles))<self.flowers:
                self.placedTiles.append(False)
                
    def placeSkull(self):
        if len(self.placedTiles)<self.totalTiles:
            if self.hasSkull & (sum(self.placedTiles)==0):
                self.placedTiles.append(True)
                
    def reset(self):
        self.placedTiles=[]
                
    def removeRandom(self):
        if self.hasSkull:
            if random.choice([True]+[False]*self.flowers):
                self.skull+=-1
                self.hasSkull=False
            else:
                self.flowers+=-1
        else:
            self.flowers+=-1
        self.totalTiles+=-1
        self.reset()

Game glass

In [ ]:
class skullGame:
    def __init__(self,player1,player2,player3,player4):
        self.playerArray = [player1,player2,player3,player4]

## Testing zone

In [125]:
x=skullPlayer()

In [126]:
x.summary()

Flowers: 3
Skull: 1
Point: 0


In [127]:
x.placeFlower()
x.placedTiles

[False]

In [128]:
x.placeSkull()
x.placedTiles

[False, True]

In [129]:
x.placeSkull()
x.placedTiles

[False, True]

In [130]:
x.removeRandom()

In [131]:
x.summary()

Flowers: 2
Skull: 1
Point: 0


In [132]:
x.placedTiles

[]

In [139]:
l=[skullPlayer() for x in range(0,4)]

In [140]:
l

In [141]:
l[0].removeRandom()

In [142]:
l[0].summary()

Flowers: 2
Skull: 1
Point: 0


In [143]:
l[1].summary()

Flowers: 3
Skull: 1
Point: 0
